In [1]:
import numpy as np
import os
import math
import sys

In [2]:

# Try to import Numba for C++ like speed.
# If not installed, it falls back to standard Python (which is slower).
try:
    from numba import jit
    HAS_NUMBA = True
except ImportError:
    HAS_NUMBA = False
    # Dummy decorator if numba is not present
    def jit(nopython=True):
        def decorator(func):
            return func
        return decorator


In [3]:

print(f"Numba acceleration: {'ENABLED' if HAS_NUMBA else 'DISABLED (Install numba for faster execution)'}")

@jit(nopython=True)
def compute_residual_jit(v, rho, N, delta, eps):
    """
    Computes the max residual.
    Optimized with JIT if available.
    """
    max_res = 0.0
    # Create a zero array for R (to mimic the passed reference in C++)
    R = np.zeros_like(v)

    for i in range(1, N - 1):
        for j in range(1, N - 1):
            res = (v[i+1, j] - 2*v[i, j] + v[i-1, j]) / (delta**2) + \
                  (v[i, j+1] - 2*v[i, j] + v[i, j-1]) / (delta**2) + \
                  rho[i, j] / eps
            R[i, j] = res
            max_res = max(max_res, abs(res))

    return max_res, R


Numba acceleration: DISABLED (Install numba for faster execution)


In [4]:

@jit(nopython=True)
def sor_iteration_jit(v, rho, N, delta, eps, omega, use_neumann_right):
    """
    Performs a single iteration of the SOR update loop.
    This is separated to allow Numba to optimize the heavy lifting.
    """
    # Neumann BC: Right boundary (x_max, corresponding to index N-1)
    # The C++ code sets v[N-1][j] = v[N-2][j]
    if use_neumann_right:
        for j in range(N):
            v[N-1, j] = v[N-2, j]

    # Main SOR Loop
    # Note: We iterate 1 to N-1 (exclusive of N-1)
    for i in range(1, N - 1):
        for j in range(1, N - 1):
            v[i, j] = (1 - omega) * v[i, j] + (omega / 4.0) * (
                v[i+1, j] + v[i-1, j] + v[i, j+1] + v[i, j-1] +
                (rho[i, j] * delta**2) / eps
            )
    return v


In [7]:

@jit(nopython=True)
def compute_energy_jit(v, rho, N, delta, eps):
    """
    Computes the energy functional S.
    """
    S = 0.0
    for i in range(1, N - 1):
        for j in range(1, N - 1):
            Ex = -(v[i+1, j] - v[i-1, j]) / (2 * delta)
            Ey = -(v[i, j+1] - v[i, j-1]) / (2 * delta)

            term = (0.5 * (Ex**2 + Ey**2) - v[i, j] * rho[i, j] / eps) * (delta**2)
            S += term
    return S

def run_sor(N, L, eps, Arho, k1, k2, k3, k4, omega, TOL, Kmax,
            use_neumann_right, potential_file, residual_file, convergence_file=""):

    delta = 2.0 * L / (N - 1)
    Vmax = 1.0

    # Grid setup
    x = np.linspace(-L, L, N)
    y = np.linspace(-L, L, N)

    # Initialize Potentials (v) and Density (rho)
    v = np.zeros((N, N))
    rho = np.zeros((N, N))

    # --- Setup rho ---
    # Using loops to match C++ logic exactly, though numpy vectorization is possible
    for i in range(N):
        for j in range(N):
            rho[i, j] = Arho * x[i] * y[j] * math.exp(-(x[i]**2 + y[j]**2))

    # --- Dirichlet BCs ---
    # C++: v[0][j] (Left/x_min), v[N-1][j] (Right/x_max)
    # C++: v[i][0] (Bottom/y_min), v[i][N-1] (Top/y_max)

    for j in range(N):
        # x = -L (index 0)
        v[0, j] = Vmax * math.sin(k1 * math.pi * (y[j] + L) / (2 * L))
        # x = +L (index N-1)
        v[N-1, j] = Vmax * math.sin(k3 * math.pi * (y[j] + L) / (2 * L))

    for i in range(N):
        # y = -L (index 0)
        v[i, 0] = Vmax * math.sin(k4 * math.pi * (x[i] + L) / (2 * L))
        # y = +L (index N-1)
        v[i, N-1] = Vmax * math.sin(k2 * math.pi * (x[i] + L) / (2 * L))

    # Open convergence file if requested
    conv_data = []

    s_old = 0.0
    s_new = 0.0

    print(f"Starting SOR... (omega={omega})")

    # --- SOR Loop ---
    for it in range(1, Kmax + 1):

        # Run Iteration (JIT compiled for speed if numba installed)
        v = sor_iteration_jit(v, rho, N, delta, eps, omega, use_neumann_right)

        # Calculate Energy Functional S
        s_new = compute_energy_jit(v, rho, N, delta, eps)

        # Check Convergence
        if it == 1:
            change = abs(s_new - s_old)
        else:
            # Avoid division by zero if s_new is 0 (unlikely but safe)
            change = abs((s_new - s_old) / s_new) if s_new != 0 else 0

        if convergence_file:
            conv_data.append(f"{it},{s_new},{change}\n")

        # Logging
        if it % 500 == 0 or it == 1:
            print(f"iter {it}   S = {s_new:.6f}   change = {change:.4e}")

        if it > 1 and change < TOL:
            print(f"Converged at iteration {it} (change={change:.4e})\n")
            break

        s_old = s_new

    # --- Compute Residuals ---
    max_res, R = compute_residual_jit(v, rho, N, delta, eps)
    print(f"Max residual = {max_res}\n")

    # --- File Output ---
    # Python generic write to match C++ output format: x, y, value
    # Flattening logic for writing
    print(f"Writing {potential_file}...")
    with open(potential_file, 'w') as f:
        for i in range(N):
            for j in range(N):
                f.write(f"{x[i]},{y[j]},{v[i, j]}\n")

    print(f"Writing {residual_file}...")
    with open(residual_file, 'w') as f:
        for i in range(N):
            for j in range(N):
                f.write(f"{x[i]},{y[j]},{R[i, j]}\n")

    if convergence_file and conv_data:
        with open(convergence_file, 'w') as f:
            f.writelines(conv_data)

    return s_new



In [8]:

def main():
    N = 100
    L = 4.0
    eps = 1.0
    TOL = 1e-8
    Kmax = 10000

    # Task 1
    print("\n--- Task 1 ---")
    run_sor(N, L, eps, 0.0, 1, -1, 1, -1, 1.5, TOL, Kmax, False,
            "potential_task1.csv", "residual_task1.csv")

    # Task 2
    print("\n--- Task 2 ---")
    run_sor(N, L, eps, 0.0, 1, -1, 1, -1, 1.5, TOL, Kmax, True,
            "potential_task2.csv", "residual_task2.csv")

    # Task 3
    print("\n--- Task 3 ---")
    omegas = [1.0, 1.3, 1.6, 1.9]
    for w in omegas:
        base = f"task3_omega_{w}"
        run_sor(N, L, eps, 0.0, 1, -1, 1, -1, w, TOL, Kmax, True,
                f"{base}_potential.csv", f"{base}_residual.csv", f"{base}_convergence.csv")

    # Task 4a
    print("\n--- Task 4a ---")
    run_sor(N, L, eps, 1.0, 0, 0, 0, 0, 1.5, TOL, Kmax, False,
            "potential_task4a.csv", "residual_task4a.csv")

    # Task 4b
    print("\n--- Task 4b ---")
    run_sor(N, L, eps, 1.0, 1, -1, 1, -1, 1.5, TOL, Kmax, False,
            "potential_task4b.csv", "residual_task4b.csv")

if __name__ == "__main__":
    main()


--- Task 1 ---
Starting SOR... (omega=1.5)
iter 1   S = 24.819805   change = 2.4820e+01
iter 500   S = 4.696065   change = 6.0378e-08
Converged at iteration 539 (change=9.7155e-09)

Max residual = 0.0015651185951086471

Writing potential_task1.csv...
Writing residual_task1.csv...

--- Task 2 ---
Starting SOR... (omega=1.5)
iter 1   S = 16.391259   change = 1.6391e+01
iter 500   S = 2.899805   change = 1.1362e-04
iter 1000   S = 2.825180   change = 1.8034e-05
iter 1500   S = 2.813744   change = 2.7273e-06
iter 2000   S = 2.812046   change = 3.9048e-07
iter 2500   S = 2.811812   change = 4.7778e-08
Converged at iteration 2808 (change=9.9936e-09)

Max residual = 0.0006228882407463518

Writing potential_task2.csv...
Writing residual_task2.csv...

--- Task 3 ---
Starting SOR... (omega=1.0)
iter 1   S = 17.886340   change = 1.7886e+01
iter 500   S = 3.135316   change = 1.8575e-04
iter 1000   S = 2.972647   change = 6.8863e-05
iter 1500   S = 2.897245   change = 3.7315e-05
iter 2000   S = 2.